In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *
from helper_functions_KP import *


#Instructions for the pipeline
Requires two inputs for training:
    - Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration)
    - NetsurfP and Biopython data that has been precalculated
    - X characteristics to predict

pipeline
Take mass spec spreadsheet
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration
Merge with Proteome data to get file that has
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,Sequence
Calculate protein features using biopython
Merge with NSP data to get all protein features

Split into X and Y dataset with Entries as labels

In [ ]:
### New Data workup for RFG

# Pull together Proteomic data
in_dir="Input_data/Proteomic data/abundance/"
all_file='Input_data/Proteomic data/abundance/Intensity _all20230202.xlsx'
#combine Mass Spec data input into one excel spreadsheet - Entry - Abundance labeled by NP Unique ID
#Abundance as a percent
#take files in_dir and combine then into one pandas df (raw_MS_data)
# files = os.listdir(in_dir)
# for i,f in enumerate(files):
#     if i==0:
#         raw_MS_data=pd.read_excel(in_dir+f,header=0)
#     else:
#         temp = pd.read_excel(in_dir+f,header=0)
#         raw_MS_data=raw_MS_data.merge(temp,how='outer',on='Entry')
raw_MS_data=pd.read_excel(all_file,header=0)
# melt the df to make it an accession number, NPUNID, Abundance dataset
raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'],var_name='Sample_num', value_name='Abundance')
#remove prots that were added due to merge
raw_MS_data=raw_MS_data.dropna()

In [ ]:
# raw_MS_data.shape
# print(raw_MS_data.Sample_num.unique())
# print(NPdata.Sample_num.unique())

In [ ]:

raw_MS_data.shape

In [ ]:
###Bring in controls (MS data for serums)##
#Controls is the abundance of the proteins in serum that have been observed via proteomics
#look at other sources here for human proteins, but internal sources for BALF and FBS
controls=pd.read_excel('Input_data/Proteomic data/controls_combined.xlsx',header=0)
MS_data_controls = pd.merge(raw_MS_data,controls,how='left', on='Entry')
###Bring in Uniprot_data,NSPdata and NP data##
uniprot_filepath='Input_data/BioPython_data/Combined_biopyCalcs.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
NSPfilePath='Input_data/NetSurfP_data/Combined.xlsx'
NSP_data=pd.read_excel(NSPfilePath)

In [ ]:
###Bring in NP data and merge to get complete NP dataset###
NP_filepath='Input_data/NPs/NP_Database.xlsx'
NPUNdata=pd.read_excel(NP_filepath,header=0,sheet_name='NPUNID')
NPprop=pd.read_excel(NP_filepath,header=0,sheet_name='NP_Props')
NPdata=pd.merge(NPUNdata,NPprop,how="left",on='NPID')
NPdata.dropna(inplace=True)

In [ ]:
#calculate Enrichment
#####MAYBE add binning here to to keep negative results and improve capapbilities######
MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
raw_prop_data=pd.merge(MS_data, uniprot_dat.drop_duplicates(subset=['Entry']), how='left',on='Entry')

In [ ]:
#normalize all columns by dividing by their max value

Protein_data_complete = pd.merge(raw_prop_data, NSP_data.drop_duplicates(subset=['Entry']),how='left', on='Entry') #merges netsurfp features and biopython features
Protein_data_complete.fillna(0,inplace=True)
#creates new column called asa_sum_normalized which is the asa_sum value divide by the mass of the protein
for df in [Protein_data_complete]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

data_complete= pd.merge(Protein_data_complete,NPdata,how='inner', on='Sample_num')
data_complete.drop(columns=['notes','Notes','NPUNID'],inplace=True)
#Optional here to drop all enrichment values that are NA - also have to deal with positive/negative infinity values
# data_complete.dropna(subset=['Enrichment'],inplace=True)

In [ ]:
# raw_prop_data.isna().sum().sum()
Protein_data_complete.isna().sum().sum()
# print(Protein_data_complete.shape)
# print(Protein_data_complete.dropna().shape)

In [ ]:
data_complete.fillna(0,inplace=True)
#replace -infintiy and positive infinity with -12 and 12

#####COME BACK AND FIND BETTER SOLUTION!!!##
data_complete= data_complete.replace([-np.inf],'-12')
data_complete=data_complete.replace([np.inf],'12')

In [ ]:
data_complete.shape
# data_complete.columns

In [ ]:

data_complete2=pd.get_dummies(data_complete, columns=['Core Material','Surface_Ligand'])
# print(data_complete2.columns)

In [ ]:
#set labels (what we are trying to predict) as Enrichment column
labels=data_complete2['Enrichment'].copy()
#make it one dimenisional
labels=np.ravel(labels)
#drop qualitative, not neccessary, and label columns
df=data_complete2.drop(['Entry','Sequence','NPID','Enrichment','Ligands','Protein Source','Sample_num','Unnamed: 5','Raw_FileID'],axis=1)


In [ ]:
out_name=[5,10,25,50,100,125,150,200,250,300,350,400,500]
esti=[5,10,25,50,100,125,150,200,250,300,350,400,500]
test_size=20
scores=[]
df2=df_norm[feat_list].copy()
for i in range(len(out_name)):
    scores[i]=rand_forest_reg_fit(df2,labels,out_name[i],0.20,esti[i])

a = pd.DataFrame(list(zip(esti, scores.ranking_)), columns=['number of estimators', 'accuracy'])
a.to_excel("Output_data/Estimator_Score_25 feats.xlsx")


In [ ]:
#Run recursive feature elimination to determine top features to select
#currently selecting 15 although that is arbitrary
### COME UP with better more objective way for determining number of features####
from sklearn.feature_selection import RFE

estimator = RandomForestRegressor(n_estimators=500)
selector = RFE(estimator, n_features_to_select= 25, step=5)
selector = selector.fit(df,labels)
# selector.support_


In [ ]:
from sklearn.preprocessing import StandardScaler

# df_scaled=std_scaler.fit_transform(df.to_numpy())
df_norm=df.copy()
for col in df_norm.columns:
    if col == 'asa_sum_normalized':
        continue
    df_norm[col]=df_norm[col]/df_norm[col].max()
    print(col)

from sklearn.feature_selection import RFE

estimator = RandomForestRegressor(n_estimators=500)
selector = RFE(estimator, n_features_to_select= 25, step=5)
selector = selector.fit(df,labels)
selector.support_
a=pd.DataFrame(list(zip(df.columns,selector.ranking_)), columns=['feature','selector rankings'])
a.to_excel("Output_data/FeatSelectionPost_norm.xlsx")

In [ ]:
selector.support_
feat_list=selector.get_feature_names_out()
df_norm=df_norm[feat_list]
x_train, x_test, y_train, y_test = train_test_split(df_norm,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
a=pd.DataFrame(list(zip(df_norm.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances_postnorm.xlsx")


In [ ]:
df_drop_dupsNSP=df_norm.copy()
to_drop=df_drop_dupsNSP.filter(like='total_exposed_')
df_drop_dupsNSP.drop(columns=to_drop,inplace=True)
estimator = RandomForestRegressor(n_estimators=500)
selector = RFE(estimator, n_features_to_select= 25, step=5)
selector = selector.fit(df_drop_dupsNSP,labels)
selector.support_
a=pd.DataFrame(list(zip(df_drop_dupsNSP.columns,selector.ranking_)), columns=['feature','selector rankings'])
a.to_excel("Output_data/FeatSelectiondrop_total_exposed.xlsx")
feat_list=selector.get_feature_names_out()
df_drop_dupsNSP=df_drop_dupsNSP[feat_list]
x_train, x_test, y_train, y_test = train_test_split(df_drop_dupsNSP,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
a=pd.DataFrame(list(zip(df_drop_dupsNSP.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances_dropNSPdupes.xlsx")


In [ ]:
df_norm

In [ ]:
feat_list=selector.get_feature_names_out()
df_rfe=df[feat_list].copy()
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
a=pd.DataFrame(list(zip(df_rfe.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances_postnorm.xlsx")


In [ ]:
feat_list=selector.get_feature_names_out()
# df_rfe.drop(['Raw_FileID'],inplace=True,axis=1)
df_rfe
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)


In [ ]:
a=pd.DataFrame(list(zip(df_rfe.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances.xlsx")

In [ ]:
#predict on every feature individually
score_list=[]
rfg=RandomForestRegressor(n_estimators=100)
for i, col in enumerate(df.columns):
    tmp=df[col].to_numpy()
    tmp=tmp.reshape(-1,1)
    x_train, x_test, y_train, y_test = train_test_split(tmp,labels, test_size = 0.20)
    rfg.fit(x_train,y_train)
    true_score = rfg.score(x_test,y_test)
    score_list.append(round(true_score,4))
    # print(i)
print(i,'columns predicted/tested')


In [ ]:
scoremax=max(score_list)
collist=list(df.columns)
maxcol=collist[(score_list.index(scoremax))]
print('max score',scoremax,maxcol)
impcols=[]
impscore=[]
feats=[]
score=[]
for i,col in enumerate(df.columns):
    feats.append(col)
    score.append(score_list[i])
    if score_list[i]> 0.45:
        impcols.append(col)
        impscore.append(score_list[i])




In [ ]:
a=pd.DataFrame(list(zip(feats,score)), columns=['feature','score'])
a.to_excel("Output_data/Individual_feat_importance.xlsx")

In [ ]:
df_top=df[feat_list].copy()
df_top.drop(['Raw_FileID'],inplace=True,axis=1)
print(df.shape)
x_train, x_test, y_train, y_test = train_test_split(df_top,labels, test_size = 0.10)

In [ ]:
# test top 50 features as selected by Recursive Feature Elimination
# determine how important each feature is by scrambling that specific feature and seeing how the prediction accuracy decreases as a result
rfg=RandomForestRegressor(n_estimators=500)
rfg.fit(x_train,y_train)
true_score = rfg.score(x_test,y_test)

feats=[]
score=[]
feats.append('score with no scrambling')
score.append(true_score)
for i, col in enumerate(x_test.columns):
    tmp=x_test.copy()
    tmp[col]=np.random.permutation(x_test[col].values)
    scram_score=(true_score-rfg.score(tmp,y_test))/true_score*100
    feats.append(col)
    score.append(round(scram_score,4))



In [ ]:
a=pd.DataFrame(list(zip(feats,score)), columns=['feature','score'])
a.to_excel("Output_data/Loss_duetoscramble.xlsx")



In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

In [ ]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)

In [ ]:
feat_list=selector.get_feature_names_out()
df_rfe=df[feat_list].copy()
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
a=pd.DataFrame(list(zip(df_rfe.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances_postnorm.xlsx")


In [ ]:
feat_list=selector.get_feature_names_out()
# df_rfe.drop(['Raw_FileID'],inplace=True,axis=1)
df_rfe
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)


In [ ]:
a=pd.DataFrame(list(zip(df_rfe.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances.xlsx")

In [ ]:
#predict on every feature individually
score_list=[]
rfg=RandomForestRegressor(n_estimators=100)
for i, col in enumerate(df.columns):
    tmp=df[col].to_numpy()
    tmp=tmp.reshape(-1,1)
    x_train, x_test, y_train, y_test = train_test_split(tmp,labels, test_size = 0.20)
    rfg.fit(x_train,y_train)
    true_score = rfg.score(x_test,y_test)
    score_list.append(round(true_score,4))
    # print(i)
print(i,'columns predicted/tested')


In [ ]:
scoremax=max(score_list)
collist=list(df.columns)
maxcol=collist[(score_list.index(scoremax))]
print('max score',scoremax,maxcol)
impcols=[]
impscore=[]
feats=[]
score=[]
for i,col in enumerate(df.columns):
    feats.append(col)
    score.append(score_list[i])
    if score_list[i]> 0.45:
        impcols.append(col)
        impscore.append(score_list[i])




In [ ]:
a=pd.DataFrame(list(zip(feats,score)), columns=['feature','score'])
a.to_excel("Output_data/Individual_feat_importance.xlsx")

In [ ]:
df_top=df[feat_list].copy()
df_top.drop(['Raw_FileID'],inplace=True,axis=1)
print(df.shape)
x_train, x_test, y_train, y_test = train_test_split(df_top,labels, test_size = 0.10)

In [ ]:
# test top 50 features as selected by Recursive Feature Elimination
# determine how important each feature is by scrambling that specific feature and seeing how the prediction accuracy decreases as a result
rfg=RandomForestRegressor(n_estimators=500)
rfg.fit(x_train,y_train)
true_score = rfg.score(x_test,y_test)

feats=[]
score=[]
feats.append('score with no scrambling')
score.append(true_score)
for i, col in enumerate(x_test.columns):
    tmp=x_test.copy()
    tmp[col]=np.random.permutation(x_test[col].values)
    scram_score=(true_score-rfg.score(tmp,y_test))/true_score*100
    feats.append(col)
    score.append(round(scram_score,4))



In [ ]:
a=pd.DataFrame(list(zip(feats,score)), columns=['feature','score'])
a.to_excel("Output_data/Loss_duetoscramble.xlsx")



In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

In [ ]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)

In [111]:
feat_list=selector.get_feature_names_out()
# df_rfe.drop(['Raw_FileID'],inplace=True,axis=1)
df_rfe
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
feat_importances=rfg.feature_importances_
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)


0.6895447495328173
frac_aa_A 2.974639083064486
frac_aa_C 2.7178849656239232
frac_aa_I 2.3205492592220955
frac_aa_R 2.808479549993177
flexibility_min 3.5805655005479524
isoelectric_point 4.654393836586511
gravy 2.5066496410522188
fraction_exposed_nonpolar_exposed 2.45279599254974
fraction_total_exposed_E 3.411109973051929
fraction_total_exposed_K 4.295110086399459
fraction_total_exposed_M 2.5947221725408762
fraction_total_exposed_R 5.770905774293792
fraction_total_exposed_Y 5.878245763326322
fraction_exposed_exposed_K 2.5940590643218626
fraction_exposed_exposed_L 5.081983975975696
fraction_exposed_exposed_M 2.4302488677261342
fraction_exposed_exposed_R 4.251238711417701
fraction_exposed_exposed_S 3.6728440357155203
nsp_secondary_structure_coil 3.271074662651876
nsp_secondary_structure_helix 3.511825970017547
asa_sum_normalized 2.9401350995789484
Zeta Potential 14.517614793655941
Dh_functionalized 7.225007620472766
Incubation Concentration (mg/ml) 4.537915600213534


In [113]:
a=pd.DataFrame(list(zip(df_rfe.columns,feat_importances*100)), columns=['feature','feat_importance'])
a.to_excel("Output_data/Featimportances.xlsx")

In [84]:
#predict on every feature individually
score_list=[]
rfg=RandomForestRegressor(n_estimators=100)
for i, col in enumerate(df.columns):
    tmp=df[col].to_numpy()
    tmp=tmp.reshape(-1,1)
    x_train, x_test, y_train, y_test = train_test_split(tmp,labels, test_size = 0.20)
    rfg.fit(x_train,y_train)
    true_score = rfg.score(x_test,y_test)
    score_list.append(round(true_score,4))
    # print(i)
print(i,'columns predicted/tested')


120 columns predicted/tested


In [87]:
scoremax=max(score_list)
collist=list(df.columns)
maxcol=collist[(score_list.index(scoremax))]
print('max score',scoremax,maxcol)
impcols=[]
impscore=[]
feats=[]
score=[]
for i,col in enumerate(df.columns):
    feats.append(col)
    score.append(score_list[i])
    if score_list[i]> 0.45:
        impcols.append(col)
        impscore.append(score_list[i])




max score 0.5055 fraction_exposed_exposed_L


In [89]:
a=pd.DataFrame(list(zip(feats,score)), columns=['feature','score'])
a.to_excel("Output_data/Individual_feat_importance.xlsx")

In [97]:
df_top=df[feat_list].copy()
df_top.drop(['Raw_FileID'],inplace=True,axis=1)
print(df.shape)
x_train, x_test, y_train, y_test = train_test_split(df_top,labels, test_size = 0.10)

(5698, 121)


In [102]:
# test top 50 features as selected by Recursive Feature Elimination
# determine how important each feature is by scrambling that specific feature and seeing how the prediction accuracy decreases as a result
rfg=RandomForestRegressor(n_estimators=500)
rfg.fit(x_train,y_train)
true_score = rfg.score(x_test,y_test)

feats=[]
score=[]
feats.append('score with no scrambling')
score.append(true_score)
for i, col in enumerate(x_test.columns):
    tmp=x_test.copy()
    tmp[col]=np.random.permutation(x_test[col].values)
    scram_score=(true_score-rfg.score(tmp,y_test))/true_score*100
    feats.append(col)
    score.append(round(scram_score,4))



In [103]:
a=pd.DataFrame(list(zip(feats,score)), columns=['feature','score'])
a.to_excel("Output_data/Loss_duetoscramble.xlsx")



In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

In [ ]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)

array([-2.5320414762967736, -2.100316148783357, -2.1346770999965723, ...,
       1.789022535723233, '-12', '-12'], dtype=object)

In [15]:
df

,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,frac_aa_G,frac_aa_H,frac_aa_I,...,Incubation Time (minutes),Temperature,Core Material_E171,Core Material_Iron Oxide,Core Material_P25,Core Material_Polystyrene,Core Material_R101,Ligand_Amine,Ligand_Carboxylate BSA,Ligand_none
0,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,0.028007,0.028007,0.024712,...,30.0,25.0,0,1,0,0,0,0,1,0
1,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,0.028007,0.028007,0.024712,...,30.0,25.0,0,1,0,0,0,0,1,0
2,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,0.028007,0.028007,0.024712,...,30.0,25.0,0,1,0,0,0,0,1,0
3,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,0.028007,0.028007,0.024712,...,30.0,25.0,0,0,0,1,0,1,0,0
4,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,0.028007,0.028007,0.024712,...,30.0,25.0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,0.048544,0.036408,0.048544,...,60.0,25.0,0,0,0,0,1,0,0,1
3078,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,0.048544,0.036408,0.048544,...,60.0,25.0,0,0,1,0,0,0,0,1
3079,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,0.048544,0.036408,0.048544,...,60.0,25.0,0,0,1,0,0,0,0,1
3080,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,0.048544,0.036408,0.048544,...,60.0,25.0,0,0,1,0,0,0,0,1


In [53]:
data_complete

KeyboardInterrupt: 

In [66]:
#Run recursive feature elimination to determine top features to select
#currently selecting 15 although that is arbitrary
### COME UP with better more objective way for determining number of features####
from sklearn.feature_selection import RFE

estimator = RandomForestRegressor(n_estimators=100)
selector = RFE(estimator, n_features_to_select= 15, step=5)
selector = selector.fit(df,labels)
selector.support_

array([False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True,  True, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [20]:
selector.ranking_

['Mass' 'frac_aa_I' 'frac_aa_Q' 'frac_aa_W' 'flexibility_std'
 'flexibility_min' 'flexibility_median' 'rsa_std'
 'fraction_total_exposed_A' 'fraction_total_exposed_Y'
 'fraction_exposed_exposed_H' 'fraction_exposed_exposed_L'
 'fraction_exposed_exposed_M' 'Sample_num' 'Raw_FileID']


In [67]:
feat_list=selector.get_feature_names_out()
print(feat_list)

['Mass' 'frac_aa_C' 'frac_aa_Q' 'fraction_exposed_nonpolar_exposed'
 'fraction_exposed_polar_exposed' 'fraction_total_exposed_E'
 'fraction_total_exposed_L' 'fraction_total_exposed_M'
 'fraction_total_exposed_Q' 'fraction_exposed_exposed_A'
 'fraction_exposed_exposed_P' 'nsp_secondary_structure_sheet'
 'nsp_secondary_structure_helix' 'Raw_FileID' 'Zeta Potential']


In [68]:
df_rfe=df[feat_list].copy()
df_rfe

,Mass,frac_aa_C,frac_aa_Q,fraction_exposed_nonpolar_exposed,fraction_exposed_polar_exposed,fraction_total_exposed_E,fraction_total_exposed_L,fraction_total_exposed_M,fraction_total_exposed_Q,fraction_exposed_exposed_A,fraction_exposed_exposed_P,nsp_secondary_structure_sheet,nsp_secondary_structure_helix,Raw_FileID,Zeta Potential
0,69293.0,0.057661,0.032949,0.257396,0.742604,0.088962,0.021417,0.003295,0.026359,0.082840,0.062130,0.005,0.710,65136,-38.0
1,38419.0,0.038997,0.047354,0.510490,0.489510,0.047354,0.061281,0.002786,0.044568,0.087413,0.118881,0.256,0.120,65136,-38.0
2,46104.0,0.014423,0.031250,0.288793,0.711207,0.060096,0.031250,0.009615,0.031250,0.060345,0.043103,0.286,0.257,65136,-38.0
3,52347.0,0.019355,0.034409,0.308824,0.691176,0.068817,0.030108,0.004301,0.027957,0.040441,0.051471,0.267,0.241,65136,-38.0
4,140374.0,0.066343,0.045307,0.327566,0.672434,0.080097,0.025890,0.004854,0.045307,0.034602,0.062284,0.391,0.005,65136,-38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5693,0.0,0.000000,0.000000,0.410853,0.589147,0.075221,0.030973,0.013274,0.022124,0.069767,0.077519,0.177,0.279,65138,-38.0
5694,60682.0,0.001873,0.037453,0.368098,0.631902,0.063670,0.018727,0.013109,0.033708,0.061350,0.085890,0.090,0.442,65138,-38.0
5695,51907.0,0.032854,0.032854,0.424920,0.575080,0.057495,0.034908,0.004107,0.030801,0.083067,0.105431,0.257,0.140,65138,-38.0
5696,20746.0,0.005525,0.044199,0.281250,0.718750,0.066298,0.016575,0.011050,0.038674,0.031250,0.031250,0.204,0.337,65138,-38.0


In [22]:
for col in df.columns:

    x_train, x_test, y_train, y_test = train_test_split(df[col],labels, test_size = 0.33, random_state=42)

0.4570470777946193


In [69]:

x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

0.6465667833499533


0.43730337548323595


In [71]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i]*100)

feature: importance score
Mass 6.219611866425997
frac_aa_C 3.7906416395642863
frac_aa_Q 5.353379181900639
fraction_exposed_nonpolar_exposed 6.581614366243876
fraction_exposed_polar_exposed 5.513655378393571
fraction_total_exposed_E 3.613280819316462
fraction_total_exposed_L 5.922764031610669
fraction_total_exposed_M 5.817975298836434
fraction_total_exposed_Q 5.086106650778767
fraction_exposed_exposed_A 5.4386754748774395
fraction_exposed_exposed_P 5.198639653426508
nsp_secondary_structure_sheet 7.66734410793521
nsp_secondary_structure_helix 4.031894860062419
Raw_FileID 22.10910602196721
Zeta Potential 7.655310648660528
